In [ ]:
import plotly.express as px
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os,sys
import folium
import branca.colormap as cm

cwd = os.getcwd()

dataloc = os.path.join("..","data","SA1_2021_AUST_SHP_GDA2020","SA1_2021_AUST_GDA2020.shp")
sa1 = gpd.read_file(dataloc)
# sa1.set_index("SA1_CODE21",inplace=True)
# fig = px.choropleth()


In [ ]:
sa1cleaned = sa1[~pd.isnull(sa1.geometry)]
sa1cleaned = sa1cleaned[sa1cleaned.STE_NAME21=="New South Wales"]

In [ ]:
sa3s = sa1cleaned.dissolve(by='SA3_CODE21', aggfunc={"AREASQKM21":'sum','SA1_CODE21':'first','SA2_NAME21':'first'})

In [ ]:
sa3s

In [ ]:
x_map=sa3s.to_crs('+proj=cea').centroid.to_crs(sa3s.crs).x.mean()
y_map=sa3s.to_crs('+proj=cea').centroid.to_crs(sa3s.crs).y.mean()
print(x_map,y_map)

In [ ]:
mymap = folium.Map(location=[y_map, x_map], zoom_start=7,tiles=None)
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(mymap)
myscale = (sa3s['AREASQKM21'].quantile((0,0.1,0.75,0.9,0.98,1))).tolist()

folium.Choropleth(
 geo_data=sa3s,
 name='Choropleth',
 data=sa3s,
 columns=['SA1_CODE21','AREASQKM21'],
 key_on="feature.properties.SA1_CODE21",
 fill_color='RdBu',
 threshold_scale=myscale,
 fill_opacity=0.5,
 line_opacity=0.2,
 legend_name="Area sq. kms",
 smooth_factor=0
).add_to(mymap)

style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
NIL = folium.features.GeoJson(
    sa3s,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['SA2_NAME21','AREASQKM21'],
        aliases=['Suburb: ','Area: '],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
mymap.add_child(NIL)
mymap.keep_in_front(NIL)
folium.LayerControl().add_to(mymap)


In [ ]:
mymap.save("output.html")